In [1]:
import torch
import numpy as np
from nilearn import plotting
import clinicadl
import pandas as pd
import matplotlib.pyplot as plt

/network/lustre/dtlake01/aramis/users/sasha.collin/miniconda3/envs/venv/lib/python3.7/site-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
input_type = ''
network_type = ''
caps_directory = ''
preprocessing = ''
tsv_path = ''

In [3]:
file = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021/subjects/sub-ADNI002S0295/ses-M00/deeplearning_prepare_data/image_based/t1_linear/sub-ADNI002S0295_ses-M00_T1w_space-MNI152NLin2009cSym_desc-Crop_res-1x1x1_T1w.pt'

In [4]:
ex = torch.load(file)

In [5]:
df = pd.read_csv('/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021.tsv',sep='\t',nrows=10)

In [6]:
df

,participant_id,session_id,alternative_id_1,sex,education_level,age_bl,marital_status,ethnic_cat,adni_rid,site,...,t1-freesurfer_segmentation-volumes_ROI-SubCortGray_volume,t1-freesurfer_segmentation-volumes_ROI-TotalGray_volume,t1-freesurfer_segmentation-volumes_ROI-SupraTentorial_volume,t1-freesurfer_segmentation-volumes_ROI-SupraTentorialNotVent_volume,t1-freesurfer_segmentation-volumes_ROI-SupraTentorialNotVentVox_volume,t1-freesurfer_segmentation-volumes_ROI-Mask_volume,t1-freesurfer_segmentation-volumes_ROI-EstimatedTotalIntraCranialVol_volume,t1-freesurfer_atlas-aicha_ROI-rh_G_Frontal_Sup-1-R_thickness,t1-freesurfer_atlas-aicha_ROI-rh_S_Sup_Temporal-1-R_thickness,t1-freesurfer_atlas-aicha_ROI-lh_G_Temporal_Pole_Mid-2-L_thickness
0,sub-ADNI002S0295,ses-M00,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,53015.0,591835.289674,1.001946e+06,959358.354552,957666.0,1619185.0,1.620886e+06,NaN,NaN,NaN
1,sub-ADNI002S0295,ses-M06,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,52945.0,591040.384576,1.006658e+06,964204.620957,962597.0,1628060.0,1.628684e+06,NaN,NaN,NaN
2,sub-ADNI002S0295,ses-M12,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,53287.0,598083.469096,1.000587e+06,957548.812134,956125.0,1616025.0,1.620672e+06,3.437,NaN,NaN
3,sub-ADNI002S0295,ses-M24,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,51883.0,573976.767801,9.810912e+05,935707.243685,934184.0,1599438.0,1.611748e+06,3.301,NaN,NaN
4,sub-ADNI002S0295,ses-M36,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,51014.0,575302.177171,9.807431e+05,933747.132027,931862.0,1593579.0,1.607849e+06,3.291,NaN,NaN
5,sub-ADNI002S0295,ses-M48,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,51687.0,559469.281730,9.914037e+05,942943.701780,940980.0,1600215.0,1.616100e+06,3.221,3.774,NaN
6,sub-ADNI002S0295,ses-M18,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,sub-ADNI002S0295,ses-M30,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,sub-ADNI002S0295,ses-M42,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,sub-ADNI002S0295,ses-M54,002_S_0295,M,18,84.8,Married,Not Hisp/Latino,295,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Train Single CNN

In [3]:
from clinicadl.tools.tsv.data_split import create_split
from clinicadl.tools.deep_learning.data import generate_sampler, return_dataset, MRIDatasetImage, get_transforms
from torch.utils.data import DataLoader
from clinicadl.tools.deep_learning.cnn_utils import train, get_criterion, test
from clinicadl.tools.deep_learning.models.random import RandomArchitecture

In [16]:
# global parameters
caps_directory = '/network/lustre/dtlake01/aramis/datasets/adni/caps/caps_v2021/'
batch_size = 1
num_workers = 1
loss = 'default'
optimizer_name = 'Adam'
learning_rate = 1e-4
weight_decay = 1e-4

In [7]:
# fold iterator
fold_iterator = range(2)

# load dataframes
AD = pd.read_csv('subjects/AD.tsv',sep='\t')
CN = pd.read_csv('subjects/CN.tsv',sep='\t')

# split data between training and validation sets
training_df, valid_df = create_split('AD', AD, 'diagnosis',0.2)
df_CN = create_split('CN', CN, 'diagnosis',0.2)
training_df = training_df.append(df_CN[0]).reset_index().iloc[np.array([0,1,2,-1,-2,-3])]
valid_df = valid_df.append(df_CN[1]).reset_index().iloc[np.array([0,1,2,-1,-2,-3])]

# drop index column
training_df.drop(columns = ['index'], inplace=True)
valid_df.drop(columns = ['index'], inplace=True)

train_transforms, all_transforms = get_transforms('image',
                                                  minmaxnormalization=False,
                                                  data_augmentation=None )

DEBUG:root:p_age=0.52, p_sex=0.9114
DEBUG:root:p_age=0.55, p_sex=0.9393
DEBUG:root:p_age=0.91, p_sex=0.9672
INFO:root:Split for diagnosis AD was found after 3 trials
DEBUG:root:p_age=0.19, p_sex=0.9074
DEBUG:root:p_age=0.73, p_sex=0.8933
DEBUG:root:p_age=0.07, p_sex=0.9819
DEBUG:root:p_age=0.79, p_sex=0.8684
DEBUG:root:p_age=0.56, p_sex=0.9432
DEBUG:root:p_age=0.01, p_sex=0.9182
DEBUG:root:p_age=0.61, p_sex=0.9819
DEBUG:root:p_age=0.77, p_sex=0.8437
DEBUG:root:p_age=0.39, p_sex=0.9819
DEBUG:root:p_age=0.37, p_sex=0.8337
DEBUG:root:p_age=0.69, p_sex=0.7854
DEBUG:root:p_age=0.54, p_sex=0.9931
DEBUG:root:p_age=0.85, p_sex=0.9074
INFO:root:Split for diagnosis CN was found after 13 trials


In [10]:
# follow structure of ``train_single_cnn``

# dataset iterator
data_train = MRIDatasetImage(caps_directory, training_df, train_transformations=all_transforms)
data_valid = MRIDatasetImage(caps_directory, valid_df, train_transformations=all_transforms)
# sampler
train_sampler=generate_sampler(data_train)
# loaders
train_loader = DataLoader(data_train,
                         batch_size=batch_size,
                         sampler=train_sampler,
                         num_workers=num_workers,
                         pin_memory=True)

valid_loader = DataLoader(data_valid,
                         batch_size=batch_size,
                         sampler=train_sampler,
                         num_workers=num_workers,
                         pin_memory=True)

# initialize the model
convolutions = {
    'conv0':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':16,
        'd_reduction': "MaxPooling"
    },
    'conv1':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':32,
        'd_reduction': "MaxPooling"
    },
    'conv2':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':32,
        'd_reduction': "MaxPooling"
    },
    'conv3':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':64,
        'd_reduction': "MaxPooling"
    },
    'conv4':{
        'n_conv':1,
        'in_channels':None,
        'out_channels':128,
        'd_reduction': "MaxPooling"
    }
}
model = RandomArchitecture(convolutions,1,data_train.size)


# Define criterion and optimizer
criterion = get_criterion(loss)
optimizer = getattr(torch.optim, optimizer_name)(filter(lambda x: x.requires_grad, model.parameters()),
                                                   lr=learning_rate,
                                                   weight_decay=weight_decay)

In [14]:
class Options:
    def __init__(self):
        self.beginning_epoch = 0
        self.tolerance = 0
        self.patience = 10
        self.epochs = 10
        self.gpu = True
        self.batch_size = 1

In [17]:
train(model, train_loader, valid_loader, criterion, optimizer,False,'log_dir','model_dir', Options())

INFO:root:Beginning epoch 0.


KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/network/lustre/dtlake01/aramis/users/sasha.collin/miniconda3/envs/venv/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1625, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1632, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: 3

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/network/lustre/dtlake01/aramis/users/sasha.collin/miniconda3/envs/venv/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    # Main proc